Example of SDSS spectra extraction

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from astropy.io import fits
# from astropy import units as u
# import pandas as pd

# path = r'E:\databases\SDSS\FITS\spec-0282-51658-0080.fits'
# hdul = fits.open(path)
# hdul.info()

In [ ]:
# cols1 = hdul[1].columns
# print(cols1)

In [ ]:
# data1 = hdul[1].data
 
# wavelength = 10 ** data1.field('loglam')
# flux = data1.field('flux')
# model = data1.field('model')
 
# print('Wavelength :', wavelength)
# print('Flux (raw) :', flux)
# print('Model (fit):', model)

In [ ]:
# wavelength = wavelength * u.Unit('AA')
# flux = flux * 10**-17 * u.Unit('erg cm-2 s-1 AA-1')
# model = model * 10**-17 * u.Unit('erg cm-2 s-1 AA-1')

In [ ]:
# fig, ax = plt.subplots(2, 1, figsize=(12,10))
 
# ax[0].plot(wavelength, flux, linewidth=1)
# ax[1].plot(wavelength, model, linewidth=1)
 
# ax[0].set_xlabel('Wavelength')
# ax[1].set_xlabel('Wavelength')
# ax[0].set_ylabel('Flux (raw)')
# ax[1].set_ylabel('Flux (fit)')
 
# plt.show()

In [ ]:
# cols3 = hdul[3].columns
# data3 = hdul[3].data
 
# print(cols3)

In [ ]:
# line_names = data3.field('LINENAME')
# line_waves = data3.field('LINEWAVE')
# line_z = data3.field('LINEZ')
# line_area = data3.field('LINEAREA')
 
# df = pd.DataFrame(
#     {'name': [i for i in line_names],
#      'lam_rest': [i for i in line_waves],
#      'z': [i for i in line_z],
#      'area': [i for i in line_area]
#      }
#     )
 
# df = df[abs(df['area']) != 0]
# df.reset_index(inplace=True)
# print(df)

In [ ]:
# df['lam_obs'] = df['lam_rest'] * (1 + df['z'])

In [ ]:
# fig, ax = plt.subplots(figsize=(12,8))
 
# label_y = np.random.uniform(
#     low=model.min().value*1.1,
#     high=model.max().value*0.9,
#     size=len(df)
#     )
 
# ax.plot(wavelength, model, linewidth=1)
 
# for i in range(len(df)):
#     ax.axvline(x=df['lam_obs'].iloc[i],
#                color='r',
#                alpha=0.3,
#                label=df['name'].iloc[i],
#                ls='--',
#                lw=0.7)
     
#     ax.text(x=df['lam_obs'].iloc[i] ,
#             y=label_y[i],
#             s=df['name'].iloc[i],
#             fontsize='small',
#             rotation=90)
 
# # plt.xlim(3700, 9300)
# plt.show()

H BETA PROCESSING

In [ ]:
import pandas as pd
import os
import sdss
from __sdss__ import *
import requests

In [ ]:
path_to_broads = r'E:\backup\backup_BPT\Sep2023\BROAD\broads_hb.csv'
frame = pd.read_csv(path_to_broads, usecols=['SPECID'])

bpt_whan_path = r'E:/backup/backup_BPT/GAMA_ETG_OLA.csv'
BPT_WHAN = pd.read_csv(bpt_whan_path, usecols=['SPECID', 'CATAID_1', 'Z', 'BPT', 'WHAN'])
BPT_WHAN = BPT_WHAN[BPT_WHAN.SPECID.isin(frame['SPECID']) == False]

BPT_WHAN = BPT_WHAN.query('WHAN == "SFG" | BPT == "UNCXY"')
BPT_WHAN.reset_index(inplace=True, drop=True)
frame = BPT_WHAN
print(frame.shape)

In [ ]:
final_frame_SDSS = frame.loc[frame.SPECID.str.contains('G') == False]
path_to_dynamic = r'E:\databases\SDSS\DYNAMIC'
path_to_images = r'E:\databases\SDSS\REST_HB'
os.chdir(path_to_dynamic)
for i in range(len(final_frame_SDSS['SPECID'])):
    sp = sdss.SpecObj(final_frame_SDSS['SPECID'][i])
    try:
        sp.download_spec()
        fits_file = ''
        for file in os.listdir(path_to_dynamic):
            fits_file = file
            plotter_extractor_sdss(path_to_dynamic + '/' + file,  path_to_images + '/' + final_frame_SDSS['SPECID'][i] +'.png', 4862.682994)
        os.remove(path_to_dynamic + '/' + fits_file)
    except:
        pass

In [ ]:
final_frame_GAMA = frame.loc[frame.SPECID.str.contains('G') == True]
path_to_dynamic = r'E:\databases\SDSS\DYNAMIC'
os.chdir(path_to_dynamic)
path_to_images = r'E:\databases\SDSS\REST_HB'
for ids in final_frame_GAMA['SPECID']:
    url = 'https://www.gama-survey.org/dr3/data/spectra/gama/reduced_27/1d/'+ids+'.fit'
    r = requests.get(url, allow_redirects=True)
    filename = url.split('/')[-1]
    open(filename, 'wb').write(r.content)
    fits_file = ''
    for file in os.listdir(path_to_dynamic):
        fits_file = file
        print(file)
        plotter_extractor_gama(path_to_dynamic + '/' + file,  path_to_images + '/' + ids +'.png', 4862.682994)
    os.remove(path_to_dynamic + '/' + fits_file)    

In [ ]:
final_frame_GAMA = frame.loc[frame.SPECID.str.contains('G') == True]
os.chdir(r'E:\databases\SDSS\GAMA_SPECTRA_PNG_HB')
for ids in final_frame_GAMA['SPECID']:
    url_png = 'https://www.gama-survey.org/dr3/data/spectra/gama/reduced_27/1d/png/'+ids+'.png'
    r = requests.get(url_png, allow_redirects=True)
    filename = url_png.split('/')[-1]
    open(filename, 'wb').write(r.content)

In [ ]:
path_to_failed = r'E:\databases\SDSS\FAILED_SDSS_HB'
FAILED_FILES = os.listdir(path_to_failed)
names = [item[:-4] for item in FAILED_FILES]
NewSeries = pd.Series(names)

DataDict = {
    'SPECID' : NewSeries
}

df = pd.DataFrame(DataDict)
df.to_csv(r'E:\backup\backup_BPT\Sep2023\SDSS_Spectra\exceptions_hb.csv', index=False)

H ALPHA PROCESSING

In [ ]:
import pandas as pd
import os
import sdss
from __sdss__ import *
import requests

In [ ]:
path_to_broads = r'E:\backup\backup_BPT\Sep2023\BROAD\broads_ha.csv'
frame = pd.read_csv(path_to_broads, usecols=['SPECID'])

bpt_whan_path = r'E:/backup/backup_BPT/GAMA_ETG_OLA.csv'
BPT_WHAN = pd.read_csv(bpt_whan_path, usecols=['SPECID', 'CATAID_1', 'Z', 'BPT', 'WHAN'])
BPT_WHAN = BPT_WHAN[BPT_WHAN.SPECID.isin(frame['SPECID']) == False]

BPT_WHAN = BPT_WHAN.query('WHAN == "SFG" | BPT == "UNCXY"')
BPT_WHAN.reset_index(inplace=True, drop=True)
frame = BPT_WHAN

frame = BPT_WHAN

In [ ]:
final_frame_SDSS = frame.loc[frame.SPECID.str.contains('G') == False]
path_to_dynamic = r'E:\databases\SDSS\DYNAMIC'
path_to_images = r'E:\databases\SDSS\REST_HA'
os.chdir(path_to_dynamic)
for i in range(len(final_frame_SDSS['SPECID'])):
    sp = sdss.SpecObj(final_frame_SDSS['SPECID'][i])
    try:
        sp.download_spec()
        fits_file = ''
        for file in os.listdir(path_to_dynamic):
            fits_file = file
            plotter_extractor_sdss(path_to_dynamic + '/' + file,  path_to_images + '/' + final_frame_SDSS['SPECID'][i] +'.png', 6564.613894)
        os.remove(path_to_dynamic + '/' + fits_file)
    except:
        pass

In [ ]:
final_frame_GAMA = frame.loc[frame.SPECID.str.contains('G') == True]
path_to_dynamic = r'E:\databases\SDSS\DYNAMIC'
os.chdir(path_to_dynamic)
path_to_images = r'E:\databases\SDSS\REST_HA'
for ids in final_frame_GAMA['SPECID']:
    url = 'https://www.gama-survey.org/dr3/data/spectra/gama/reduced_27/1d/'+ids+'.fit'
    r = requests.get(url, allow_redirects=True)
    filename = url.split('/')[-1]
    open(filename, 'wb').write(r.content)
    fits_file = ''
    for file in os.listdir(path_to_dynamic):
        fits_file = file
        print(file)
        plotter_extractor_gama(path_to_dynamic + '/' + file,  path_to_images + '/' + ids +'.png', 6564.613894)
    os.remove(path_to_dynamic + '/' + fits_file)

In [ ]:
final_frame_GAMA = frame.loc[frame.SPECID.str.contains('G') == True]
os.chdir(r'E:\databases\SDSS\GAMA_SPECTRA_PNG_HA')
for ids in final_frame_GAMA['SPECID']:
    url_png = 'https://www.gama-survey.org/dr3/data/spectra/gama/reduced_27/1d/png/'+ids+'.png'
    r = requests.get(url_png, allow_redirects=True)
    filename = url_png.split('/')[-1]
    open(filename, 'wb').write(r.content)

In [ ]:
path_to_failed = r'E:\databases\SDSS\FAILED_SDSS_HA'
FAILED_FILES = os.listdir(path_to_failed)
names = [item[:-4] for item in FAILED_FILES]
NewSeries = pd.Series(names)

DataDict = {
    'SPECID' : NewSeries
}

df = pd.DataFrame(DataDict)
df.to_csv(r'E:\backup\backup_BPT\Sep2023\SDSS_Spectra\exceptions_ha.csv', index=False)

CHECKING WAVY GALAXIES

In [ ]:
import pandas as pd
import os

In [ ]:
path_to_wavies = r"E:\databases\SDSS\WAVY"
FAILED_FILES = os.listdir(path_to_wavies)
names = [item[:-4] for item in FAILED_FILES]
NewSeries = pd.Series(names)

DataDict = {
    'SPECID' : NewSeries
}
df_wavies = pd.DataFrame(DataDict)

bpt_whan_path = r'E:/backup/backup_BPT/GAMA_ETG_OLA.csv'
BPT_WHAN = pd.read_csv(bpt_whan_path, usecols=['SPECID', 'CATAID_1', 'Z', 'BPT', 'WHAN'])

NewD = pd.merge(df_wavies, BPT_WHAN, how='inner', on='SPECID')
# NewD = BPT_WHAN

specall_path = r"E:\databases\GAMAs\SpecAll.txt"
SpecAllDict = {
    0 : 'SPECID',
    3 : 'RA',
    4 : 'DEC',
    7 : 'Z',
    11 : 'URL',
    12 : 'URL_IMG',
    13 : 'CATAID_1',
    16 : 'DIST',
    17 : 'IS_SBEST',
    18 : 'IS_BEST'
}

SpecAll = pd.read_csv(specall_path, sep=r"\s+", header=None, index_col=0, engine='python', usecols=SpecAllDict.keys(), names=SpecAllDict.values())
# print(SpecAll)

NewDD = pd.merge(NewD, SpecAll, how='inner', on='CATAID_1')
NewDD.to_csv(r'E:\backup\backup_BPT\Sep2023\SDSS_Spectra\waves.csv', index=False)